# Introduction

We propose to achieve movie genre classification based only on movie poster images. A deep neural network is constructed to jointly describe visual appearance and object information, and classify a given movie poster image into genres. Because a movie may belong to multiple genres, this is a multi-label image classification problem. To facilitate related studies, we collect a large-scale movie poster dataset, associated with various metadata. Based on this dataset, we fine-tune a pre-trained convolutional neural network to extract visual representation and adopt a state-of-the-art framework to detect objects in posters. Two types of information are then integrated by the proposed neural network. In the evaluation, we show that the proposed method yields encouraging performance, which is much better than previous works. 



In [0]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
     |████████████████████████████████| 450kB 50.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


# Multi-Label Classification

Each sample can belong to more than one class. The CNN will have as well 
C
 output neurons. The target vector 
t
 can have more than a positive class, so it will be a vector of 0s and 1s with 
C
 dimensionality.
This task is treated as 
C
 different binary 
(
C
′
=
2
,
t
′
=
0
 or 
t
′
=
1
)
 and independent classification problems, where each output neuron decides if a sample belongs to a class or not.
 
(![alt text](https://gombru.github.io/assets/cross_entropy_loss/multiclass_multilabel.png))

![alt text](https://)# New Section

In [0]:
import pandas as pd
import json
import glob  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [0]:
print(tf.__version__)

2.0.0


In [0]:

  from google.colab import drive
drive.mount('/content/drive')

In [0]:
from zipfile import ZipFile
# Create a ZipFile Object and load Movie_poster_dataset.zip in it
with ZipFile('/content/drive/My Drive/Movie_Poster_Dataset.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/Multiclassification_Genre/images')

In [0]:
from zipfile import ZipFile
# Create a ZipFile Object and load Movie_poster_Metadata in it
with ZipFile('/content/drive/My Drive/Movie_Poster_Metadata.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/Multiclassification_Genre/MoviesList')

In [0]:
ff=open("/content/Multiclassification_Genre/MoviesList/groundtruth/1980.txt","r")
Movies=ff.read()
comma=Movies.replace("}","},")
newc=comma[:-2]
sqaureAdd="["+newc+"]"
aaa=sqaureAdd.replace("\n","")
jj=aaa.replace("ObjectId(","").replace("\")","\"")
fin=json.loads(jj)
df = pd.DataFrame.from_dict(fin, orient='columns')

In [0]:
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
dfFin
dfFin2=dfFin

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

In [14]:
all_files = glob.glob("/content/Multiclassification_Genre/MoviesLis/*.txt")

print((all_files))
count=0
for files in all_files:
  try:
    ff=open(files,"r",encoding='utf-16')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
  except:
    ff=open(files,"r",encoding='utf-8')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
    

[]


In [15]:
Movie_data=pd.concat([dfFin,dfFin2])
#Movie_data.head()
Movie_data


NameError: ignored

Data Exploration and Pre-processing

In [0]:
#Creating a subset of the the total data

Movie_Final=Movie_data[['imdbID','Genre',]]


In [0]:
Movie_Final.to_csv('Movie_genre.csv',encoding="ISO-8859-1")

In [0]:
#Onehot encoding
data = pd.read_csv("Movie_genre.csv")
data.shape

#df.dropna(subset=['Genre'],inplace=True)
#df.drop_duplicates(subset=['imdbID'], keep='first', inplace=True)
#df.set_index('imdbID',inplace=True)
#target = df['Genre'].apply(lambda x: [w.strip() for w in x.split(',')])
#genres = pd.get_dummies(target.apply(pd.Series).stack()).sum(level=0)


(6984, 4)

In [0]:
data

In [0]:
Movie_genre=Movie_Final
Movie_genre['Genre']=[elem[0] for elem in Movie_genre['Genre']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
Moviegenrestats=Movie_genre.groupby('Genre').size().reset_index(name='counts')

In [0]:
Moviegenrestats=Moviegenrestats.sort_values('counts',ascending=False)

In [0]:
MovieFilter=list(Moviegenrestats)

In [0]:
#Getting the image Id's
image_paths = glob.glob("/content/Multiclassification_Genre/images/*.jpg")
image_ids = []
for path in image_paths:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
traindatagenerator=train_datagen.flow_from_dataframe(
    dataframe=Movie_Final,
    directory='/content/Multiclassification_Genre/images',
    x_col='imdbID',
    y_col='Genre',
    batch_size=50,
    shuffle=True)

Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 6984 invalid image filename(s) in x_col="imdbID". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [0]:
testdatagenerator=test_datagen.flow_from_dataframe(
    dataframe=testdatagen,
    directory='/content/Multiclassification_Genre/images',
    x_col='imdbID',
    y_col='Genre',
    batch_size=50,
    shuffle=True
    )

NameError: ignored